We can observe the file not contains an only one json, it contains a json for value.
We need to read line by line and manage with each json object to make a colection.

In [ ]:
import pandas as pd
import Util



values = []
time = []
f = open("../data/raw/measures/H-DHT11-measures.json", "r")
for line in f:
    json_parsed = json.loads(line)
    values.append( json_parsed['value'])
    time.append(pd.to_datetime(json_parsed['time']))

We create a Panda's Dataframe to can manage data.

In [ ]:
data_obj = {'values':values, 'time': time}
data = pd.DataFrame()
data['values'] = values
data['time'] = pd.Series(time, name="time", dtype='datetime64[s]')
print(data)

We draw a plot of data. And we can observe that this file haven't got all data. We can observe a holes.

In [ ]:
data.plot.line(x = 'time', y = 'values')

Try de find holes, by finding time difference. Make a plot where can view when data ara lost.

In [ ]:
data['timediff'] = (data['time'] -  data['time'].shift(1)).astype('timedelta64[s]')
data.plot.scatter(x = 'time', y = 'timediff')

We try to get and objective interval acceptable of losing data.

Mode is the most repeated and we could consider is the time that is programmed to get data from devices. So that is 4 seconds. We can filter the lowest times that time increase and we could fit the threshold.

In [ ]:
g = data.groupby(['timediff'])
v1 = g.count();
#print(v1)
v2 = v1[v1.values>10]
mode = v1.values.max()
print ("mode", mode)
v2.plot.line(y="values")
print(v2)
#v1.plot.line( y='values')

Try to get interval of lost data. We consider that data is lost if we don't have data during 30 seconds from previous point but we can be more hard fitting value to 7 seconds. We can observe if this threshold goes down the intervals with lost data increases a lot.
We show you all intervals

In [ ]:
threshold = 30
data_filter = data[data.timediff.gt(threshold)]

for idx in data_filter.index:
    print( "from", data['time'][idx-1], "to", data['time'][idx], "time(s) losed data", data['timediff'][idx] )

Finally, we don't need to change o delete data because this file only have problems of lose data but haven't any erroneous data.
We save in a normalized format to can be loaded easily after.

In [ ]:
data.drop(columns="timediff").to_json('../data/processed/H-DHT11-measures.json', orient='records', date_format='iso', indent=2)
